In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.models import Sequential
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Fruit Image Dataset/dataset_2.csv")
dataset.head()

,image_path,class
0,/content/drive/MyDrive/Fruit Image Dataset/1_g...,guava
1,/content/drive/MyDrive/Fruit Image Dataset/2_b...,banana
2,/content/drive/MyDrive/Fruit Image Dataset/3_p...,peaches
3,/content/drive/MyDrive/Fruit Image Dataset/4_m...,mango
4,/content/drive/MyDrive/Fruit Image Dataset/5_r...,raspberries


In [ ]:
label_encoder = OneHotEncoder(sparse_output=False)
labels_onehot = label_encoder.fit_transform(dataset[['class']])
num_classes = labels_onehot.shape[1]

In [ ]:
def load_and_preprocess_image(image_path):
  image = cv2.imread(image_path)
  image = cv2.resize(image,(100,100))
  image = image.astype("float32")/255.0
  return image
num_processes  = 5
with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
    images = list(executor.map(load_and_preprocess_image, dataset['image_path']))
images = np.array(images)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(images, labels_onehot, test_size = 0.3, random_state = 0)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip=True,
    validation_split = 0.3
)
datagen.fit(x_train)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
input_shape = [100,100,3]
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64,(3,3),activation = 'relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128,(3,3),activation = 'relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(256,(3,3),activation = 'relu',input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 128)       0

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 100),
    validation_data = (x_test,y_test),
    epochs = 30,
    steps_per_epoch = len(x_train)//100
)

Epoch 1/30
70/70 [==============================] - 32s 279ms/step - loss: 2.1357 - accuracy: 0.1750 - val_loss: 1.7310 - val_accuracy: 0.3567
Epoch 2/30
70/70 [==============================] - 23s 328ms/step - loss: 1.6971 - accuracy: 0.3434 - val_loss: 1.6123 - val_accuracy: 0.3733
Epoch 3/30
70/70 [==============================] - 19s 266ms/step - loss: 1.5119 - accuracy: 0.4321 - val_loss: 1.3179 - val_accuracy: 0.5323
Epoch 4/30
70/70 [==============================] - 20s 284ms/step - loss: 1.3885 - accuracy: 0.4829 - val_loss: 1.3704 - val_accuracy: 0.4597
Epoch 5/30
70/70 [==============================] - 19s 272ms/step - loss: 1.2773 - accuracy: 0.5194 - val_loss: 1.2972 - val_accuracy: 0.5337
Epoch 6/30
70/70 [==============================] - 19s 273ms/step - loss: 1.2441 - accuracy: 0.5479 - val_loss: 1.3748 - val_accuracy: 0.5263
Epoch 7/30
70/70 [==============================] - 20s 293ms/step - loss: 1.1089 - accuracy: 0.5996 - val_loss: 1.1228 - val_accuracy: 0.6057

In [ ]:
test_loss,test_accuracy = model.evaluate(x_test,y_test,verbose = 2)

94/94 - 1s - loss: 0.3849 - accuracy: 0.8733 - 659ms/epoch - 7ms/step


In [ ]:
y_pred_train = model.predict(x_train)
y_pred_train =  np.argmax(y_pred_train, axis=1)
y_pred_test = model.predict(x_test)
y_pred_test =  np.argmax(y_pred_test, axis=1)

94/94 [==============================] - 0s 5ms/step


In [ ]:

# Code to start the finetunning process
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

train_cm = confusion_matrix(np.argmax(y_train,axis =1),y_pred_train)
train_acc = accuracy_score(np.argmax(y_train,axis = 1),y_pred_train)
train_pre = precision_score(np.argmax(y_train,axis = 1),y_pred_train,average='weighted')
train_rscore = recall_score(np.argmax(y_train,axis =1),y_pred_train,average='weighted')
train_f1 = f1_score(np.argmax(y_train,axis =1),y_pred_train,average='weighted')

test_cm = confusion_matrix(np.argmax(y_test,axis =1),y_pred_test)
test_acc = accuracy_score(np.argmax(y_test,axis =1), y_pred_test)
test_pre = precision_score(np.argmax(y_test,axis =1),y_pred_test,average='weighted')
test_rscore = recall_score(np.argmax(y_test,axis =1),y_pred_test,average='weighted')
test_f1 = f1_score(np.argmax(y_test,axis =1),y_pred_test, average='weighted')

print("**********Training Dataset**************")
print(train_cm)
print("Accuracy = ",train_acc)
print("Precision Score = ",train_pre)
print("Recall Score = ",train_rscore)
print( "F1 Score = ",train_f1)

print("**********Testing Dataset**************")
print(test_cm)
print("Accuracy = ",test_acc)
print("Precision Score = ",test_pre)
print("Recall Score = ",test_rscore)
print("F1 Score = ",test_f1)

**********Training Dataset**************
[[597  13   4  11   5  33  10  16   7   1]
 [  2 657  22   5   2   4  13   0   0   3]
 [  0   1 685   1   0   0   0   0  21   0]
 [  3  14   7 597   0   7  18   0   0  25]
 [  0  11   0   8 677   0   2   0  13   0]
 [ 37  15   0  15  19 542  66   6   1   7]
 [  4  19   2  26   2  14 625   0   0  23]
 [ 27   0   0   0  33  11   2 605   0   1]
 [  0   0   9   0  17   0   0   0 675   1]
 [  0   0   0   2   8   2   0   0   8 681]]
Accuracy =  0.9058571428571428
Precision Score =  0.9060099088427562
Recall Score =  0.9058571428571428
F1 Score =  0.9050457263744445
**********Testing Dataset**************
[[259   5   0   4   2  14   7   5   7   0]
 [  1 259   7   8   2   5   7   0   0   3]
 [  0   1 290   0   0   0   0   0   1   0]
 [  3   9   1 272   6   3  13   0   0  22]
 [  0   9   0   3 267   3   0   0   7   0]
 [ 23  10   2  15   5 190  25  10   6   6]
 [  2   7   2  14   1   8 242   1   0   8]
 [ 23   0   0   1  25   7   1 264   0   0]
 [  0   0